# Frost Days & Mortality Days November - March

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as plt
import datetime as dt
import os
import re

In [ ]:
# All directories
root       = "C:/Users/freiste/OneDrive - Ilmatieteen laitos/Documents/IIASA YSSP 2023"
this_dir   =  os.getcwd()

input_dir  = f"{root}/02 - Data/EU/Copernicus_E-OBS_Weather_Original" # to save space, not all data might be there anymore
output_dir = f"{root}/02 - Data/EU/Copernicus_E-OBS_Weather_Postprocessed"

In [ ]:
def frost_mortality_days(yr):
    
    if 1980 <= yr <= 1994:
        strtyr = 1980
        endyr  = 1994
    elif 1995 <= yr <= 2010:
        strtyr = 1995
        endyr  = 2010
    elif 2011 <= yr <= 2022:
        strtyr = 2011
        endyr  = 2022
    else:
        print('Error, year unknown.')
    
    tmin = xr.open_dataset(f"{input_dir}/tempmin_daily_{strtyr}-{endyr}_0.25deg.nc")
    tmax = xr.open_dataset(f"{input_dir}/tempmax_daily_{strtyr}-{endyr}_0.25deg.nc")
    tmin.close()
    tmax.close()


    # Index of the overwintering period Nov-Mar
    if y == 0:
        idx1 = tmin.indexes["time"].get_loc(f"{yr}-01-01T00:00:00",  method="nearest")

    else:
        idx1 = tmin.indexes["time"].get_loc(f"{yr-1}-11-01T00:00:00",  method="nearest")

    idx2 = tmin.indexes["time"].get_loc(f"{yr}-03-31T00:00:00",  method="nearest")

    # Filter out overwintering period temperatures
    tmin = tmin.tn.isel(time=slice(idx1, idx2+1))
    tmax = tmax.tx.isel(time=slice(idx1, idx2+1))


    # frost and mortality days
    frostdays     = tmax.where(tmax <= -1).count('time')
    frostdays     = frostdays.where(frostdays>14)   

    mortalitydays = tmin.where(tmin <= -15).count('time')
    mortalitydays = mortalitydays.where(mortalitydays>0)

    frostdays     = (frostdays-mortalitydays )
    mortalitydays = mortalitydays.where(mortalitydays>30)
    
    result = frostdays.to_dataset(name='frostdays')
    result['overkilldays'] = mortalitydays
    result = result.assign_coords({'year' : yr})

    return result


In [ ]:
yrs = range(1980,2023)

for y, yr in enumerate(yrs):
    
    if y==0:
        overwintering_mortality_days = frost_mortality_days(yr)
    else:
        overwintering_mortality_days = xr.concat([overwintering_mortality_days, frost_mortality_days(yr)], dim="year")

        
overwintering_mortality_days = \
overwintering_mortality_days.assign_attrs({'frostdays'    : 'Days on which maximum ground temperature was below or equal to -1 degrees Celsius. Grid points are only considered if there were a minimum of 14 frost days during the cold season Nov-Mar. We assume 50% overwintering mortality for bark beetles.',
                                           'overkilldays' : 'Days on which minimum ground temperature was below or equal to -15 degrees Celsius. Grid points are only considered if there were a minimum of 30 overkill days during the cold season Nov-Mar. We assume 100% overwintering mortality for bark beetles.',
                                           'background mortality': 'For overwintering in warmer grid points, we assume the background mortality of 20%.'})


# Export frost days data
overwintering_mortality_days.to_netcdf(f"{output_dir}/overwintering_mortality_europe_1980-2022_0.25deg.nc")
overwintering_mortality_days

# Now gotta select lat and long from it

In [ ]:
# Beetle overwintering mortality in iLand, fd = frost days
M = 1−exp^(-0.1005)*fd